1er Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

importo las librerias

In [64]:
import pandas as pd
import re
import numpy as np


Parametros


In [100]:
fecha_inicio = '01/10/2024'
fecha_final = '14/10/2024'
filtros = "DISPONIBLE ESTADISTICA"

lugar_catalogados= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

unidades_municipios = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA"
}

provincias = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}

nombre_archivo = 'Planilla.xlsx'
nombre_hojas =['GEOG. PROCEDIMIENTOS','VEHI. Y PERSO. CONTROLADAS','PERSONAL Y ELEMENTOS AFECTADOS', 'DETENIDOS Y APRENDIDOS','INCAUTACIONES','TRATA O TRAFIC PERSONAS','OTROS DELITOS','OTROS EVENTOS', 'FALLECIDOS','ABATIDOS']


Cargo los archivos excel de las carpetas bajadas

In [131]:

# Carga el archivo de Excel
wb_base = pd.read_excel('data/Base de datos.xlsx')
wb_procedimientos = pd.read_excel('bajadas/bajada_general.xls')
wb_arma = pd.read_excel('bajadas/bajada_arma.xls')
wb_vehiculos_secuestros = pd.read_excel('bajadas/bajada_vehiculos_secuestrados.xls')
wb_vehiculos_secuestros_ministerio = pd.read_excel('bajadas/bajada_ministerio_vehiculos.xls')
wb_personas = pd.read_excel('bajadas/bajada_general_persona.xls')
wb_secuestradas = pd.read_excel('bajadas/bajada_divisas_secuestradas.xls')
wb_sustancia = pd.read_excel('bajadas/bajada_general_narcotrafico.xls')

Filtrar lo que ya se informaron en la base de datos y lo que estan disponible en estadistica

In [ ]:
wb_procedimientos = wb_procedimientos[wb_procedimientos['ESTADO_PARTE'] == 'DISPONIBLE ESTADISTICA'].copy()
print (wb_procedimientos.count())

Funciones

In [126]:
def procesar_causa_judicial(row):
    causa = row['CAUSAJUDICIALNUMERO']
    if pd.isna(causa) or causa in ["", "S/D", "A/S", "N/C"]:
        return f"{row['TIPO_CAUSA_INTERNA']} {row['CAUSA_INTERNA_NUMERO']}".strip()
    
    causa_str = str(causa).strip()
    
    prefijos = ["NRO.", "N°", "EXPT", "EXPEDIENTE", "CAUSA"]
    
    for prefijo in prefijos:
        if causa_str.upper().startswith(prefijo):
            causa_str = causa_str[len(prefijo):].strip()
    
    # Eliminar cualquier carácter que no sea número, letra o /ML
    causa_str = re.sub(r'[^\w/]', '', causa_str)
    
    return causa_str



def procesar_descripcion(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if tipo == "DENUNCIA":
        return "DENUNCIA POLICIAL"
    elif tipo == "CONTROL PREVENTIVO":
        return f"CONTROL PREVENTIVO - {procesar_lugar(row)}"
    elif tipo == "ORDEN DE ALLANAMIENTO":
        return "ORDEN DE ALLANAMIENTO"
    elif tipo == "ORDEN DE ALLANAMIENTO / DETENCIÓN":
        return "ORDEN DE ALLANAMIENTO"
    else:
        return "OTRO MANDATO JUDICIAL"


def procesar_tipo(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if pd.isna(tipo):
        return ""
    elif tipo == "DENUNCIA"  or tipo == "CONTROL PREVENTIVO" :
        return "ORDEN POLICIAL"
    else:
        return "ORDEN JUDICIAL"
    
def procesar_provincia(row):
    provincia = row['PROVINCIA']
    if pd.isna(provincia):
        return ""
    return provincias.get(provincia, provincia)
    
def procesar_municipio(row):
    unidad = row['UOSP']
    if pd.isna(unidad):
        return ""
    return unidades_municipios.get(unidad, unidad)

def procesar_lugar(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    return  lugar_catalogados[lugar]

def procesar_direccion(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    if lugar == "FUERA DE JURISDICCION":
        return str(row['CALLE']) + " " + str(row['NUMERO']) + ", " + str(row['CIUDAD']) + " - " + str(row['PARTIDO'])
    else:
        return ""



Pruebas

In [130]:

wb_nueva_hoja = wb_procedimientos.copy()


wb_nueva_hoja['FUERZA_INTERVINIENTE'] = 'PSA'
wb_nueva_hoja['ID_OPERATIVO'] = wb_procedimientos.apply(procesar_causa_judicial, axis=1)
wb_nueva_hoja['ID_PROCEDIMIENTO'] = wb_nueva_hoja.index + 1
wb_nueva_hoja['UNIDAD_INTERVINIENTE'] = wb_procedimientos['UOSP']
wb_nueva_hoja['DESCRIPCIÓN'] = wb_procedimientos.apply(procesar_descripcion, axis=1)
wb_nueva_hoja['TIPO_INTERVENCION'] = wb_procedimientos.apply(procesar_tipo, axis=1)
wb_nueva_hoja['PROVINCIA'] = wb_procedimientos.apply(procesar_provincia, axis=1)
wb_nueva_hoja['DEPARTAMENTO O PARTIDO'] = wb_procedimientos.apply(procesar_municipio, axis=1)
wb_nueva_hoja['LOCALIDAD'] = "-"
wb_nueva_hoja['DIRECCION'] = wb_procedimientos.apply(procesar_direccion, axis=1)
wb_nueva_hoja['ZONA_SEGURIDAD_FRONTERAS'] = "-"
wb_nueva_hoja['PASO_FRONTERIZO'] = "-"
wb_nueva_hoja['LATITUD'] = wb_procedimientos['GEOREFERENCIA_Y']
wb_nueva_hoja['LONGITUD'] = wb_procedimientos['GEOREFERENCIA_X']
wb_nueva_hoja['OTRAS AGENCIAS INTERVINIENTES'] = "-"
wb_nueva_hoja['Observaciones - Detalles'] = "-"

# Separar fecha y hora de la columna DENUNCIAFECHA
wb_nueva_hoja['FECHA'] = pd.to_datetime(wb_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
wb_nueva_hoja['HORA'] = pd.to_datetime(wb_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')

print (wb_nueva_hoja.head(10))
hoja_procedimientos = wb_nueva_hoja[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO','UNIDAD_INTERVINIENTE','DESCRIPCIÓN','TIPO_INTERVENCION','PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION', 'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD','FECHA', 'HORA','OTRAS AGENCIAS INTERVINIENTES','Observaciones - Detalles']]

hoja_procedimientos.to_excel("Planilla.xlsx", sheet_name='GEOG. PROCEDIMIENTOS', index=False)

print(f"\nArchivo Excel '{nombre_archivo}' creado con éxito.")

  URSA UOSP CAUSA_INTERNA_NUMERO     TIPO_PARTE AMPLIACION  NUMERO_PARTE  \
0  RG1  EZE          0379-EZE/24  PARTE URGENTE        ---          4150   
1  RG3  SAL          0056-SAL/24  PARTE URGENTE        ---          4154   
2  RG1  EZE          0261-EZE/24  PARTE URGENTE        ---          4155   
3  RG4  NaN          0119-RG4/24  PARTE URGENTE        ---          4156   
4  RG5  BAR          0139-BAR/24  PARTE URGENTE        ---          4157   
5  RG2  COR          0126-COR/24  PARTE URGENTE        ---          4158   
6  RG5  NEU          0139-NEU/24  PARTE URGENTE        ---          4160   
7  RG2  COR          0127-COR/24  PARTE URGENTE        ---          4161   
8  RG1  MDP          0008-MDP/24  PARTE URGENTE        ---          4162   
9  RG5  NEU          0140-NEU/24  PARTE URGENTE        ---          4163   

   ANIO_PARTE      FECHA_CREACION    TIPO_HECHO         CLASIFICACION_NIVEL_1  \
0        2024 2024-10-01 12:01:36     DELICTIVO   DELITOS CONTRA LA PROPIEDAD   
1

Control de salidad de la informacion

In [ ]:
# Aplicar la función procesar_causa_judicial a la copia

# # Contar cuántos casos se procesaron vs. cuántos se mantuvieron igual
# procesados = (wb_nueva_hoja['CAUSAJUDICIALNUMERO'] != wb_nueva_hoja['ID_OPERATIVO']).sum()
# print(f"\nCasos procesados: {procesados}")
# print(f"Casos sin cambios: {len(wb_nueva_hoja) - procesados}")